In [1]:
# Basic imports
import os
import time
import sys
from IPython.display import clear_output
import random
import numpy as np

import pandas as pd
import math


# Tensorflow and Keras
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

# Image processing
from PIL import Image
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

2024-04-18 05:55:57.057544: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 05:55:57.214393: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-18 05:55:57.977852: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-04-18 05:55:57.978122: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [3]:
import torch

# Set seeds to ensure reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)  # For multi-GPU setups

In [4]:
# Set paths
base_data_path = (
    "../../aws_s3/messidor_224_Crop/"  # <-- Add path to the numpy files here
)
output_path = "output/"  # <-- Add path to the output folder here

In [5]:
# decide if we want to do validation or only Train-Test
validation_flag = True

In [6]:
# Augmentation config

# decide whether to apply augmentation or not
apply_augmentation = True

# decide whether to apply brightness augmentation or not
aug_br_flag = 1  # 1 = Augment Brightness, anything else = No Brightness Augmentation

In [7]:
# Cross Testing

# Decide if we are evaluating only 1 dataset or cross testing with other dataset as well
cross_testing = True

cross_data_path = "../../aws_s3/Idrid_224_Crop/"  # <-- Add path to the numpy files here

In [10]:
# Configure GPU memory growth
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Set memory growth before initializing GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(
            "Num GPUs Available: Physical GPUs=",
            len(gpus),
            " | Logical GPUs=",
            len(logical_gpus),
        )
    except RuntimeError as e:
        # Handle potential errors here
        print(e)

In [11]:
## Augemntation Helper Fucntions
## image is an np array of the image
def rotate_image(image, angle):
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image


def flip_image(image):
    flip_variations = [
        (False, False),  # No flip
        (True, False),  # Horizontal flip
        (False, True),  # Vertical flip
        (True, True),  # Both flips
    ]

    random.shuffle(flip_variations)
    augmented_images = []

    for flip_horizontal, flip_vertical in flip_variations[:4]:
        if flip_horizontal and flip_vertical:
            augmented_images.append(cv2.flip(image, -1))  # horizontal and vertical
        elif flip_horizontal:
            augmented_images.append(cv2.flip(image, 1))  # horizontal
        elif flip_vertical:
            augmented_images.append(cv2.flip(image, 0))  # vertical
        else:
            augmented_images.append(image)  # No flip

    return augmented_images


def shear_image(image, shear_range):
    height, width = image.shape[:2]
    shear_value = random.uniform(-shear_range, shear_range)

    if shear_value < 0:
        shear_matrix = np.array([[1, -shear_value, 0], [0, 1, 0]])
    else:
        shear_matrix = np.array([[1, shear_value, 0], [0, 1, 0]])

    sheared_image = cv2.warpAffine(image, shear_matrix, (width, height))
    return sheared_image


def translate_image(image, translate_range):
    height, width = image.shape[:2]

    max_shift_x = int(width * 0.1)
    max_shift_y = int(height * 0.1)

    translate_x = random.randint(-max_shift_x, max_shift_x)
    translate_y = random.randint(-max_shift_y, max_shift_y)

    translation_matrix = np.array(
        [[1, 0, translate_x], [0, 1, translate_y]], dtype=np.float32
    )

    translated_image = cv2.warpAffine(image, translation_matrix, (width, height))

    return translated_image


def adjust_brightness(image, brightness_range):
    brightness_factor = 1.0 + random.uniform(-brightness_range, brightness_range)
    adjusted_image = np.clip(image * brightness_factor, 0.25, 255).astype(np.uint8)
    return adjusted_image

In [12]:
## image parameter is an np array of an image
## flag 1: includes brightness, 0 or any other value exludes it


def augmented_fn2(image, flag):
    augmented_images = []

    for _ in range(4):
        angle = random.uniform(-35, 35)
        image_r = rotate_image(image, angle)
        augmented_images.append(image_r)

    image_f = flip_image(image)
    augmented_images.extend(image_f)

    for _ in range(4):
        image_s = shear_image(image, shear_range=0.15)
        augmented_images.append(image_s)

    for _ in range(4):
        image_t = translate_image(image, translate_range=0.1)
        augmented_images.append(image_t)

    if flag == 1:
        for _ in range(4):
            image_b = adjust_brightness(image, brightness_range=0.25)
            augmented_images.append(image_b)

    return np.array(augmented_images)

In [13]:
# Load the base data
X_train = np.load(base_data_path + "X_train.npy")
y_train = np.load(base_data_path + "y_train.npy")
X_test = np.load(base_data_path + "X_test.npy")
y_test = np.load(base_data_path + "y_test.npy")

In [14]:
# load the cross data if cross testing is enabled
if cross_testing:
    X_cross = np.load(cross_data_path + "X_test.npy")
    y_cross = np.load(cross_data_path + "y_test.npy")

In [ ]:
# Augmentation
if apply_augmentation:
    # temporarily store the augmented data
    X_train_aug = []
    y_train_aug = []

    for i in range(len(X_train)):
        X_train_aug.extend(augmented_fn2(X_train[i], aug_br_flag))
        y_train_aug.extend([y_train[i]] * 20)

    X_train = np.array(X_train_aug)
    y_train = np.array(y_train_aug)

In [ ]:
# data validation

assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]
assert X_train.shape[1:] == X_test.shape[1:]

# Dimension check, for EfficientNetB0, the input shape is (224, 224, 3)
assert X_train.shape[1:] == (224, 224, 3)

# if cross testing is enabled, validate the cross data
if cross_testing:
    assert X_cross.shape[1:] == (224, 224, 3)
    assert X_cross.shape[0] == y_cross.shape[0]

print("Data loaded and augmented successfully, the data shapes are:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
if cross_testing:
    print("X_cross shape:", X_cross.shape)
    print("y_cross shape:", y_cross.shape)

In [ ]:
# splitting train data into train and validation if validation flag is enabled
if validation_flag:
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42
    )

In [15]:
import tensorflow as tf
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    LearningRateScheduler,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import SGD, Adam
import numpy as np
import cv2
import random

# Load pre-trained MobileNetV2 model without the top classification layer
base_model = MobileNetV2(
    weights="imagenet", include_top=False, input_shape=(224, 224, 3)
)

# Freeze the base model layers
base_model.trainable = False


# Reducing dropout rate
model = Sequential(
    [
        base_model,
        Flatten(),
        # Dense(1024, activation="relu"),
        # BatchNormalization(),
        # Dropout(0.4),  # Adjusted dropout rate
        Dense(512, activation="relu"),
        BatchNormalization(),
        Dropout(0.4),  # Adjusted dropout rate
        Dense(256, activation="relu"),
        BatchNormalization(),
        Dropout(0.4),  # Adjusted dropout rate
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(0.4),  # Adjusted dropout rate
        Dense(4, activation="softmax"),
    ]
)

2024-04-18 06:04:16.207201: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
# Modifying learning rate scheduler
def exponential_decay(epoch, lr):
    initial_learning_rate = 1e-1
    decay_rate = 0.9
    decay_steps = 10
    lr = initial_learning_rate * decay_rate ** (epoch / decay_steps)
    return lr


# Define optimizer
optimizer = SGD(learning_rate=1e-1)

# Compile the model
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

In [ ]:
batch_size = 4
## converting the data to tf dataset for optimization and deleting the numpy arrays to free up memory

if validation_flag:
    # Converting validation data to tf data
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(batch_size)
    # del X_val, y_val

# converting test data to tf dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(batch_size)
del X_test, y_test

# Converting trainig Data to tf dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

# Shuffle and batch the dataset
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
del X_train, y_train


# if cross testing is enabled, convert the cross data to tf dataset
if cross_testing:
    cross_dataset = tf.data.Dataset.from_tensor_slices((X_cross, y_cross)).batch(
        batch_size
    )
    del X_cross, y_cross

In [ ]:
# callbacks
early_stopping = EarlyStopping(
    restore_best_weights=True,
    monitor="val_loss",
    patience=10,
)

if validation_flag:
    # training the model
    history = model.fit(
        # X_train_tensor,
        # y_train_tensor,
        train_dataset,
        epochs=20,
        validation_data=val_dataset,
        # batch_size=batch_size,
        callbacks=[LearningRateScheduler(exponential_decay)],
    )
else:
    history = model.fit(
        # X_train_tensor,
        # y_train_tensor,
        train_dataset,
        epochs=20,
        # batch_size=batch_size,
        callbacks=[LearningRateScheduler(exponential_decay)],
    )

In [ ]:
# evaluating the model
test_loss_self, test_acc_self = model.evaluate(test_dataset)

if cross_testing:
    test_loss_cross, test_acc_cross = model.evaluate(cross_dataset)
    print(
        f"Test Accuracies are: Self: {test_acc_self:.2%} | Cross: {test_acc_cross:.2%} | Difference= {test_acc_self - test_acc_cross:.2%}"
    )
else:
    print(f"Test Accuracy self:{test_acc_self:.2%}")